+++++++++++++++++++++++++++

This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0

*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]

+++++++++++++++++++++++++++

### This notebook is an exemplar which demonstrates transferring files from a Box folder to a users home directory on Savio. 

( tested with boxsdk (2.0.0a2) on python 3.5 kernel)
pip install -Iv boxsdk==2.0.0a2 

function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [ ]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

Oauth2 information is read from a local file with three lines, one line per parameter. 
The client id and client secret are defined in the Box application created for this notebook.  Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens that are then continually refreshed.

In [ ]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None
os.chdir('/global/home/users/mmanning')
# Read app info from text file
with open('app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


The refresh token is read from a local file.
This token was created by running the bootstrap notebook which requires the user to validate
with CalNet Authentication Service credentials, then stores the returned auth and refresh tokens 
in the same config files.

In [ ]:
REFRESH_TOKEN = None

# Read app info from text file
with open('apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

Perform autentication then create globus client
Verify client is working by retrieving the name of the users root folder in Box

In [ ]:
from boxsdk import OAuth2
from boxsdk import Client

# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
#print ("items: ", items )



get the items from the base folder. currently the Box SDK does not have an option for finding a folder by name so if you are looking for a specific folder then you would need to loop thru all the items in the list below and do a name match. Once you find the folder and retrieve the id, you can save that id for subsequent runs. I have not found a way to get the id of the folder from the Box web client.

In [ ]:
import os
import shutil 

print ('current working directory: ', os.getcwd())
os.chdir('/global/scratch/user_name_here/test')

# test folder contents
items = client.folder(folder_id='0').get_items(limit=20, offset=0)
if type(items) is list:
    print ('number of files in top folder: ', len(items) )
    
    for item in items:
        if item['type'] == 'folder':
            print('folder name: ', item['name'])
        # download all image files
        if  not item['type'] == 'folder' and item['name'].endswith('.jpg'):
            imagecontent = client.file(file_id=item['id']).content()
            newfile = open('/global/scratch/user_name_here/' + item['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()

Create a new folder in the base directory and upload image files in the current folder.

In [ ]:
newFolder = client.folder(folder_id='0').create_subfolder('ThisIsATest')
newFolderId = newFolder['id']
sourceFolder = '/global/scratch/user_name_here/'

print ("new folder id: ", newFolderId )

upload_folder = client.folder(folder_id=newFolderId).get()

# upload all the files in the current folder  if os.path.isfile(f)
files = [f for f in os.listdir(sourceFolder) ] 
print ("files: ", files )

for filename in files:
    print ("file name: ", filename ) 
    
    if filename.endswith('.jpg'):
        upload_folder.upload(sourceFolder + filename)  
